In [22]:
import pandas as pd
from sqlite3 import connect

In [23]:
sumo = pd.read_csv("results.csv", dtype={'basho': 'string', 'day': 'string', 'rikishi1_id': 'int64', 'rikishi1_rank': 'string', 'rikishi1_shikona': 'string', 'rikishi1_result': 'string', 'rikishi1_win': 'int64', 'kimarite': 'string', 'rikishi2_id': 'int64', 'rikishi2_rank': 'string', 'rikishi2_shikona': 'string', 'rikishi2_result': 'string', 'rikishi2_win': 'string'})
banzuke = pd.read_csv("banzuke.csv", dtype={'basho': 'string', 'day': 'string', 'rikishi1_id': 'int64', 'rikishi1_rank': 'string', 'rikishi1_shikona': 'string', 'rikishi1_result': 'string', 'rikishi1_win': 'int64', 'kimarite': 'string', 'rikishi2_id': 'int64', 'rikishi2_rank': 'string', 'rikishi2_shikona': 'string', 'rikishi2_result': 'string', 'rikishi2_win': 'string'})


In [24]:
connection = connect(':memory:')
sumo.to_sql("sumo", connection)
banzuke.to_sql("banzuke", connection)
def sql(a_string):
    return(pd.read_sql(a_string, connection))

In [25]:
combined_sql_query = pd.read_sql_query(
    """SELECT sumo.basho AS 'basho', sumo.rikishi1_id AS 'id', sumo.rikishi1_shikona, banzuke.rikishi as 'rikishi', 
    sumo.rikishi1_rank AS 'rank', banzuke.height AS 'height', banzuke.weight AS 'weight', banzuke.heya AS 'heya',
    banzuke.shusshin AS 'shusshin', sumo.rikishi1_win as 'outcome', sumo.kimarite AS 'kimarite', sumo.rikishi2_id AS 'opponent id',
    sumo.rikishi2_shikona AS 'opponent'
    FROM sumo 
    JOIN banzuke 
    ON sumo.rikishi1_shikona = banzuke.rikishi 
    AND sumo.basho = banzuke.basho""",
    connection,
)

combined_sql_query
combined_df = pd.DataFrame(combined_sql_query)
combined_df = combined_df.drop(["rikishi1_shikona"], axis=1)
combined_df

,basho,id,rikishi,rank,height,weight,heya,shusshin,outcome,kimarite,opponent id,opponent
0,1983.01,4140,Chikubayama,J13w,175.5,115.0,Miyagino,Fukuoka,0,yorikiri,4306,Ofuji
1,1983.01,4306,Ofuji,Ms1e,193.5,145.0,Kokonoe,Tokyo,1,yorikiri,4140,Chikubayama
2,1983.01,1337,Tochitsukasa,J12w,178.5,130.0,Kasugano,Aichi,1,oshidashi,4323,Shiraiwa
3,1983.01,4323,Shiraiwa,J13e,182.4,103.0,Isegahama,Akita,0,oshidashi,1337,Tochitsukasa
4,1983.01,4097,Tamakiyama,J12e,182.0,134.0,Kataonami,Fukuoka,0,yorikiri,4319,Harunafuji
...,...,...,...,...,...,...,...,...,...,...,...,...
227938,2023.07,11985,Daieisho,S1w,182.0,161.0,Oitekaze,Saitama,0,hikiotoshi,11855,Takanosho
227939,2023.07,12451,Hoshoryu,S1e,186.0,131.0,Tatsunami,Mongolia,1,uwatenage,12796,Hakuoho
227940,2023.07,12796,Hakuoho,M17w,179.0,156.0,Miyagino,Tottori,0,uwatenage,12451,Hoshoryu
227941,2023.07,12094,Abi,K1w,188.0,149.0,Shikoroyama,Saitama,1,yorikiri,12231,Kirishima


In this next cell, I am trying to figure out a way to change 'rank' to be a string, but this is proving difficult so far.

In [6]:
combined_df[combined_df['rank'].str.startswith('Y')]  

,id,rikishi,rank,weight
58,4095,Wakanohana,Y2eHD,133.0
61,4080,Kitanoumi,Y1w,165.0
62,1354,Chiyonofuji,Y1e,116.0
123,4095,Wakanohana,Y2eHD,133.0
124,1354,Chiyonofuji,Y1e,116.0
...,...,...,...,...
227347,11927,Terunofuji,Y1e,173.0
227387,11927,Terunofuji,Y1e,173.0
227428,11927,Terunofuji,Y1e,173.0
227467,11927,Terunofuji,Y1e,173.0


In [7]:
# combined_df.loc[combined_df.rikishi == 'Takamisakari']
# combined_df.loc[combined_df.weight == 130]
# combined_df.loc[combined_df.rikishi.str.contains('Koto')]
# 

combined_df["rank"] = combined_df["rank"].astype(str)

combined_df.loc[combined_df["rank"].str.contains('Y')]

,id,rikishi,rank,weight
58,4095,Wakanohana,Y2eHD,133.0
61,4080,Kitanoumi,Y1w,165.0
62,1354,Chiyonofuji,Y1e,116.0
123,4095,Wakanohana,Y2eHD,133.0
124,1354,Chiyonofuji,Y1e,116.0
...,...,...,...,...
227347,11927,Terunofuji,Y1e,173.0
227387,11927,Terunofuji,Y1e,173.0
227428,11927,Terunofuji,Y1e,173.0
227467,11927,Terunofuji,Y1e,173.0


In [8]:
rikishi_list = []
for i in combined_df["rikishi"].unique():
    rikishi_list.append(i)

rikishi_list

['Chikubayama',
 'Ofuji',
 'Tochitsukasa',
 'Shiraiwa',
 'Tamakiyama',
 'Harunafuji',
 'Kotogatake',
 'Hakuryuyama',
 'Zaonishiki',
 'Hachiya',
 'Takamisugi',
 'Tengoyama',
 'Daitetsu',
 'Kakureizan',
 'Tochiakagi',
 'Shinko',
 'Itai',
 'Hakuryu',
 'Sakahoko',
 'Sanofuji',
 'Hidanohana',
 'Kotochitose',
 'Onokuni',
 'Koboyama',
 'Wakajishi',
 'Washuyama',
 'Asahifuji',
 'Banryuyama',
 'Kurosegawa',
 'Tochitsurugi',
 'Jingaku',
 'Shishiho',
 'Wakasegawa',
 'Amanoyama',
 'Tamaryu',
 'Tochihikari',
 'Tagaryu',
 'Wakanofuji',
 'Takamiyama',
 'Azumanada',
 'Hoo',
 'Saisu',
 'Ozutsu',
 'Fujizakura',
 'Masudayama',
 'Aobajo',
 'Toryu',
 'Kaiki',
 'Kirinji',
 'Sadanoumi',
 'Kurama',
 'Asashio',
 'Oshio',
 'Wakashimazu',
 'Hokutenyu',
 'Kotokaze',
 'Onishiki',
 'Takanosato',
 'Wakanohana',
 'Dewanohana',
 'Daijuyama',
 'Kitanoumi',
 'Chiyonofuji',
 'Oyutaka',
 'Dewanoshima',
 'Tejima',
 'Ichinowatari',
 'Suiryu',
 'Kotowaka',
 'Tochiizumi',
 'Hananokuni',
 'Hoshi',
 'Kinoarashi',
 'Rinho',
 'Ar

In [9]:
# There are two lines for each match, one for the winner and one for the loser.
# So as not to show duplicates of every match, we'll just show the winners with sumo_only_wins
sumo_only_wins = sumo[
    sumo.rikishi1_win == int("1")
]  # could also use sumo.loc[sumo.rikishi1_win == 1]

sumo_only_wins[["rikishi1_shikona", "kimarite"]]
# sumo_only_wins

,rikishi1_shikona,kimarite
1,Ofuji,yorikiri
2,Tochitsukasa,oshidashi
5,Harunafuji,yorikiri
7,Hakuryuyama,tsukidashi
9,Hachiya,hatakikomi
...,...,...
227763,Kotonowaka,yorikiri
227765,Asanoyama,yorikiri
227766,Takanosho,hikiotoshi
227768,Hoshoryu,uwatenage
